# Chapter 14: Advanced Rendering Techniques

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adiel2012/computer-vision/blob/main/chapter_14_advanced_rendering.ipynb)

**Advanced rendering techniques** extend basic rendering with effects like ambient occlusion, subsurface scattering, volumetric rendering, and screen-space techniques. These methods add realism and atmosphere to rendered scenes.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
import random
from typing import Tuple, List, Optional
from dataclasses import dataclass

In [ ]:
class Vec3:
    def __init__(self, x: float = 0.0, y: float = 0.0, z: float = 0.0):
        self.x, self.y, self.z = x, y, z
    
    def __add__(self, other):
        return Vec3(self.x + other.x, self.y + other.y, self.z + other.z)
    
    def __sub__(self, other):
        return Vec3(self.x - other.x, self.y - other.y, self.z - other.z)
    
    def __mul__(self, scalar: float):
        return Vec3(self.x * scalar, self.y * scalar, self.z * scalar)
    
    def __truediv__(self, scalar: float):
        return Vec3(self.x / scalar, self.y / scalar, self.z / scalar)
    
    def __neg__(self):
        return Vec3(-self.x, -self.y, -self.z)
    
    def dot(self, other) -> float:
        return self.x * other.x + self.y * other.y + self.z * other.z
    
    def cross(self, other):
        return Vec3(
            self.y * other.z - self.z * other.y,
            self.z * other.x - self.x * other.z,
            self.x * other.y - self.y * other.x
        )
    
    def length(self) -> float:
        return math.sqrt(self.dot(self))
    
    def normalize(self):
        l = self.length()
        return self / l if l > 0 else Vec3(0, 0, 0)

class Color:
    def __init__(self, r: float = 0.0, g: float = 0.0, b: float = 0.0):
        self.r, self.g, self.b = r, g, b
    
    def __add__(self, other):
        return Color(self.r + other.r, self.g + other.g, self.b + other.b)
    
    def __mul__(self, other):
        if isinstance(other, Color):
            return Color(self.r * other.r, self.g * other.g, self.b * other.b)
        return Color(self.r * other, self.g * other, self.b * other)
    
    def __truediv__(self, scalar: float):
        return Color(self.r / scalar, self.g / scalar, self.b / scalar)
    
    def clamp(self, min_val: float = 0.0, max_val: float = 1.0):
        return Color(
            max(min_val, min(max_val, self.r)),
            max(min_val, min(max_val, self.g)),
            max(min_val, min(max_val, self.b))
        )
    
    def to_tuple(self) -> Tuple[float, float, float]:
        return (self.r, self.g, self.b)

print("✓ Base classes loaded")

## 1. Ambient Occlusion (AO)

**Ambient Occlusion** approximates soft shadowing in crevices and corners.

### Theory

AO measures accessibility to ambient lighting:

$$
A(\mathbf{p}) = \frac{1}{\pi} \int_{\Omega} V(\mathbf{p}, \omega) (\mathbf{n} \cdot \omega) \, d\omega
$$

where $V(\mathbf{p}, \omega)$ is visibility (0 if occluded, 1 if visible).

### Monte Carlo Approximation

$$
A(\mathbf{p}) \approx \frac{1}{N} \sum_{i=1}^{N} V(\mathbf{p}, \omega_i)
$$

Sample random directions in hemisphere, cast rays, count occlusions.

### Screen Space Ambient Occlusion (SSAO)

Fast approximation using depth buffer:
1. Sample points around pixel in screen space
2. Compare depths to determine occlusion
3. Accumulate occlusion factor

In [ ]:
def random_in_hemisphere(normal: Vec3) -> Vec3:
    """Generate random direction in hemisphere"""
    while True:
        v = Vec3(random.uniform(-1, 1), random.uniform(-1, 1), random.uniform(-1, 1))
        if v.length() < 1 and v.dot(normal) > 0:
            return v.normalize()

def compute_ambient_occlusion(point: Vec3, normal: Vec3, scene, 
                             num_samples: int = 16, max_dist: float = 1.0) -> float:
    """Compute ambient occlusion using ray casting"""
    occlusion = 0.0
    
    for _ in range(num_samples):
        # Random direction in hemisphere
        direction = random_in_hemisphere(normal)
        
        # Cast ray
        ray_origin = point + normal * 0.001  # Offset to avoid self-intersection
        
        # Check for occlusion within max_dist
        if scene.intersect_ao_ray(ray_origin, direction, max_dist):
            occlusion += 1.0
    
    # Return accessibility (1 - occlusion)
    return 1.0 - (occlusion / num_samples)

print("✓ Ambient occlusion functions loaded")

## 2. Subsurface Scattering (SSS)

**Subsurface scattering** simulates light penetrating and scattering within translucent materials.

### Dipole Approximation

Approximate SSS using two point sources:

$$
S(r) = \frac{\alpha'}{4\pi} \left[ \frac{e^{-\sigma_t r_1}}{r_1^2} (\sigma_t r_1 + 1) - \frac{e^{-\sigma_t r_2}}{r_2^2} (\sigma_t r_2 + 1) \right]
$$

where:
- $r_1, r_2$ are distances to real and virtual sources
- $\sigma_t$ is attenuation coefficient
- $\alpha'$ is reduced albedo

### Simplified SSS

Wrap lighting to simulate subsurface:

$$
L = \max(0, \frac{(\mathbf{n} \cdot \mathbf{l}) + w}{1 + w})
$$

where $w$ is wrap amount.

In [ ]:
def subsurface_scattering_wrap(n_dot_l: float, wrap: float = 0.5) -> float:
    """Simple wrap lighting for SSS approximation"""
    return max(0.0, (n_dot_l + wrap) / (1.0 + wrap))

def translucent_shadow(thickness: float, light_intensity: float, 
                      absorption: float = 2.0) -> float:
    """Compute light transmission through translucent material"""
    # Beer-Lambert law
    transmission = math.exp(-absorption * thickness)
    return light_intensity * transmission

print("✓ Subsurface scattering functions loaded")

## 3. Volumetric Rendering

**Volumetric rendering** handles participating media like fog, smoke, and clouds.

### Volume Rendering Equation

$$
L(\mathbf{x}, \omega) = \int_0^D T(t) \sigma_s(\mathbf{x} + t\omega) L_s(\mathbf{x} + t\omega, \omega) \, dt + T(D) L_0
$$

where:
- $T(t) = e^{-\int_0^t \sigma_t(s) ds}$ is transmittance
- $\sigma_s$ is scattering coefficient
- $\sigma_t = \sigma_s + \sigma_a$ (scattering + absorption)
- $L_s$ is scattered radiance
- $L_0$ is background radiance

### Ray Marching

Discrete approximation:

$$
L \approx \sum_{i=1}^{N} T_i \cdot \sigma_s(\mathbf{x}_i) \cdot L_s(\mathbf{x}_i) \cdot \Delta t + T_N \cdot L_0
$$

where $T_i = \prod_{j=1}^{i-1} (1 - \sigma_t(\mathbf{x}_j) \Delta t)$

In [ ]:
class VolumetricMedium:
    """Homogeneous volumetric medium"""
    def __init__(self, density: float, albedo: Color, phase_g: float = 0.0):
        self.density = density  # sigma_t
        self.albedo = albedo    # sigma_s / sigma_t
        self.phase_g = phase_g  # Henyey-Greenstein parameter
    
    def phase_function(self, cos_theta: float) -> float:
        """Henyey-Greenstein phase function"""
        g = self.phase_g
        denom = 1.0 + g * g - 2.0 * g * cos_theta
        return (1.0 - g * g) / (4.0 * math.pi * denom ** 1.5)

def ray_march_volume(ray_start: Vec3, ray_dir: Vec3, t_max: float,
                    medium: VolumetricMedium, light_dir: Vec3,
                    light_color: Color, num_steps: int = 64) -> Color:
    """Ray marching through volumetric medium"""
    dt = t_max / num_steps
    
    accumulated_color = Color(0, 0, 0)
    transmittance = 1.0
    
    for i in range(num_steps):
        t = (i + 0.5) * dt
        pos = ray_start + ray_dir * t
        
        # Density at current position (could be spatially varying)
        density = medium.density
        
        # Scattering
        cos_theta = ray_dir.dot(light_dir)
        phase = medium.phase_function(cos_theta)
        
        # In-scattering contribution
        scattering = density * phase
        inscatter = light_color * (medium.albedo * scattering * transmittance * dt)
        
        accumulated_color = accumulated_color + inscatter
        
        # Update transmittance
        transmittance *= math.exp(-density * dt)
    
    return accumulated_color

print("✓ Volumetric rendering functions loaded")

## 4. Bloom and Glow

**Bloom** simulates light bleeding from bright areas.

### Algorithm

1. Extract bright pixels: $L_{bright} = \max(0, L - threshold)$
2. Blur bright pixels (Gaussian)
3. Composite: $L_{final} = L + intensity \cdot L_{blur}$

### Gaussian Blur

$$
G(x, y) = \frac{1}{2\pi\sigma^2} e^{-\frac{x^2 + y^2}{2\sigma^2}}
$$

Separable: blur horizontally then vertically for efficiency.

In [ ]:
def extract_bright_pixels(image: np.ndarray, threshold: float = 0.8) -> np.ndarray:
    """Extract pixels brighter than threshold"""
    bright = np.maximum(0, image - threshold)
    return bright

def gaussian_blur_1d(image: np.ndarray, sigma: float = 2.0, 
                    axis: int = 0) -> np.ndarray:
    """1D Gaussian blur along axis"""
    # Create Gaussian kernel
    kernel_size = int(6 * sigma + 1)
    if kernel_size % 2 == 0:
        kernel_size += 1
    
    kernel_half = kernel_size // 2
    kernel = np.zeros(kernel_size)
    
    for i in range(kernel_size):
        x = i - kernel_half
        kernel[i] = math.exp(-(x * x) / (2 * sigma * sigma))
    
    kernel /= kernel.sum()
    
    # Apply convolution
    from scipy.ndimage import convolve1d
    return convolve1d(image, kernel, axis=axis, mode='reflect')

def apply_bloom(image: np.ndarray, threshold: float = 0.8, 
               intensity: float = 0.5, blur_sigma: float = 5.0) -> np.ndarray:
    """Apply bloom effect to image"""
    # Extract bright pixels
    bright = extract_bright_pixels(image, threshold)
    
    # Blur (separable Gaussian)
    blurred = gaussian_blur_1d(bright, blur_sigma, axis=0)
    blurred = gaussian_blur_1d(blurred, blur_sigma, axis=1)
    
    # Composite
    result = image + intensity * blurred
    return np.clip(result, 0, 1)

print("✓ Bloom functions loaded")

## 5. Depth of Field (Post-Process)

**Depth of Field** can be approximated in post-processing using depth buffer.

### Circle of Confusion

$$
CoC = \frac{A |d - d_{focus}|}{d(d_{focus} - f)}
$$

where:
- $A$ is aperture
- $d$ is object distance
- $d_{focus}$ is focus distance
- $f$ is focal length

Blur radius proportional to circle of confusion.

In [ ]:
def compute_coc(depth: float, focus_distance: float, 
               aperture: float = 0.1, focal_length: float = 1.0) -> float:
    """Compute circle of confusion size"""
    if depth <= 0:
        return 0
    
    coc = aperture * abs(depth - focus_distance) / (depth * (focus_distance - focal_length))
    return min(coc, 10.0)  # Clamp max blur

print("✓ Depth of field functions loaded")

## Example 1: Ambient Occlusion Visualization

In [ ]:
# Create simple AO visualization
width, height = 400, 400
ao_image = np.ones((height, width))

# Simulate cavity - darker in corners
for j in range(height):
    for i in range(width):
        # Distance from edges
        dx = min(i, width - i) / width
        dy = min(j, height - j) / height
        dist = min(dx, dy)
        
        # Darken corners
        ao = 0.3 + 0.7 * (dist / 0.5) ** 0.5
        ao_image[j, i] = min(1.0, ao)

# Apply to surface
base_color = np.ones((height, width, 3)) * [0.8, 0.6, 0.4]
ao_applied = base_color * ao_image[:, :, np.newaxis]

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))

ax1.imshow(base_color)
ax1.set_title('Base Color')
ax1.axis('off')

ax2.imshow(ao_image, cmap='gray')
ax2.set_title('Ambient Occlusion (white=accessible)')
ax2.axis('off')

ax3.imshow(ao_applied)
ax3.set_title('With AO Applied')
ax3.axis('off')

plt.tight_layout()
plt.show()

print("Notice darker areas in corners (occluded regions).")

## Example 2: Volumetric Fog

In [ ]:
# Render scene with volumetric fog
width, height = 400, 300
fog_image = np.zeros((height, width, 3))

# Fog medium
fog = VolumetricMedium(density=0.1, albedo=Color(0.8, 0.8, 0.9), phase_g=0.3)
light_dir = Vec3(0.5, 0.7, 0.3).normalize()
light_color = Color(1.0, 0.95, 0.8)

for j in range(height):
    if j % 50 == 0:
        print(f"Row {j}/{height}")
    
    for i in range(width):
        # Ray through pixel
        u = (2.0 * i / width - 1.0)
        v = (1.0 - 2.0 * j / height)
        
        ray_start = Vec3(u * 2, v * 1.5, 3)
        ray_dir = Vec3(0, 0, -1)
        
        # Background gradient
        t = 0.5 * (v + 1.0)
        background = Color(0.5, 0.7, 1.0) * t + Color(1.0, 1.0, 1.0) * (1.0 - t)
        
        # Ray march through fog
        fog_color = ray_march_volume(ray_start, ray_dir, 6.0, fog, 
                                     light_dir, light_color, num_steps=32)
        
        # Composite with background
        final = fog_color + background * 0.3
        
        fog_image[j, i] = final.clamp(0, 1).to_tuple()

plt.figure(figsize=(10, 7))
plt.imshow(fog_image)
plt.title('Volumetric Fog using Ray Marching')
plt.axis('off')
plt.tight_layout()
plt.show()

print("Volumetric fog with light scattering.")

## Example 3: Bloom Effect

In [ ]:
# Create test image with bright spots
test_image = np.zeros((300, 400, 3))

# Add some bright lights
centers = [(100, 150), (300, 150), (200, 80), (200, 220)]
for cx, cy in centers:
    for j in range(300):
        for i in range(400):
            dist = math.sqrt((i - cx)**2 + (j - cy)**2)
            if dist < 20:
                intensity = max(0, 1.0 - dist / 20)
                test_image[j, i] = [intensity * 1.5, intensity * 1.2, intensity]

# Apply bloom
try:
    from scipy.ndimage import convolve1d
    bloomed = apply_bloom(test_image, threshold=0.6, intensity=0.8, blur_sigma=8.0)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    
    ax1.imshow(test_image)
    ax1.set_title('Original (Bright Lights)')
    ax1.axis('off')
    
    ax2.imshow(bloomed)
    ax2.set_title('With Bloom Effect')
    ax2.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print("Bloom creates glow around bright areas.")
except ImportError:
    print("scipy not available - bloom effect requires scipy.ndimage")
    plt.figure(figsize=(7, 5))
    plt.imshow(test_image)
    plt.title('Test Image (install scipy for bloom)')
    plt.axis('off')
    plt.show()

## Summary

**Advanced rendering techniques** add realism beyond basic shading:

### Techniques Covered

1. **Ambient Occlusion**
   - Contact shadows in crevices
   - Screen-space (SSAO) or ray-traced
   - Enhances depth perception

2. **Subsurface Scattering**
   - Light penetration in translucent materials
   - Essential for skin, wax, marble
   - Dipole approximation or wrap lighting

3. **Volumetric Rendering**
   - Participating media (fog, smoke, clouds)
   - Ray marching through volume
   - Phase functions for anisotropic scattering

4. **Bloom/Glow**
   - Light bleeding from bright sources
   - Extract-blur-composite pipeline
   - Simulates camera/eye response

5. **Depth of Field**
   - Focal blur effects
   - Circle of confusion
   - Can be ray-traced or post-processed

### Applications

- **Film/Animation**: Realistic lighting and atmosphere
- **Games**: Real-time approximations (SSAO, fast SSS)
- **Visualization**: Scientific rendering, medical imaging
- **VR/AR**: Immersive environments

### Performance Considerations

- **AO**: Expensive if ray-traced; SSAO is real-time friendly
- **SSS**: Full dipole is slow; approximations for real-time
- **Volumetrics**: Ray marching is costly; optimize step count
- **Post-processing**: Generally fast (bloom, DoF)

These techniques are essential for production-quality rendering.